In [6]:
import numpy as np
import torch.nn as nn
from ncps.wirings import AutoNCP
from ncps.torch import LTC
import pytorch_lightning as pl
import torch
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader

from Sophia import SophiaG

In [1]:
120 * (3.141/180)

2.094

In [3]:
# LightningModule for training a RNNSequence module
class SequenceLearner(pl.LightningModule):
    def __init__(self, model, lr=0.005):
        super().__init__()
        self.model = model
        self.lr = lr

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat, _ = self.model.forward(x)
        y_hat = y_hat.view_as(y)
        loss = nn.MSELoss()(y_hat, y)
        self.log("train_loss", loss, prog_bar=True)
        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat, _ = self.model.forward(x)
        y_hat = y_hat.view_as(y)
        loss = nn.MSELoss()(y_hat, y)

        self.log("val_loss", loss, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        # Here we just reuse the validation_step for testing
        return self.validation_step(batch, batch_idx)

    def configure_optimizers(self):
        return SophiaG(self.model.parameters(), lr=self.lr)
        # return torch.optim.Adam(self.model.parameters(), lr=self.lr)

In [4]:
input_data = np.load("../datasets/gradual_2D_hdv_randomstart_1000_5_64.npy",allow_pickle=True)

In [11]:
class Datamodel(Dataset):
    def __init__(self,time_pos_points,labels):
        self.x = time_pos_points
        self.y = labels
    def __len__(self): 
        return self.x.shape[0]
    def __getitem__(self, ix):
        return self.x[ix], self.y[ix]
    
data = input_data[:,0]
labels = input_data[:,1]
datamodel = Datamodel(data, labels)
data_loader = DataLoader(datamodel, batch_size=None, shuffle=False, num_workers=1)

In [12]:
out_features = 2
in_features = 2

# Use high precision for matrix multiplication for tensor cores
torch.set_float32_matmul_precision("high")

wiring = AutoNCP(16, out_features)  # 16 units, 1 motor neuron

ltc_model = LTC(in_features, wiring, batch_first=True)
learn = SequenceLearner(ltc_model, lr=0.005)
trainer = pl.Trainer(
    logger=pl.loggers.CSVLogger("log"),
    max_epochs=200,
    gradient_clip_val=None,  # Clip gradient to stabilize training
    accelerator="gpu",
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


alloc!


In [13]:
trainer.fit(learn, data_loader)

/home/mattis/anaconda3/envs/torch2/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | LTC  | 1.5 K 
-------------------------------
1.2 K     Trainable params
288       Non-trainable params
1.5 K     Total params
0.006     Total estimated model params size (MB)


/home/mattis/anaconda3/envs/torch2/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:   0%|          | 0/1000 [00:00<?, ?it/s] 

RuntimeError: The size of tensor a (16) must match the size of tensor b (2) at non-singleton dimension 2